In [79]:
import pandas as pd

filename ='C:/Users/User/Downloads/sap_storing_data_hu_project.csv'
df = pd.read_csv(filename, low_memory=False)
#verwijder alle kolommen die niet van gebruik zijn:
df = df.drop(columns=['stm_sap_mon_meld_ddt', 'stm_mon_begin_ddt', 'stm_mon_toelichting_trdl', 'stm_oh_pg_mld', 'stm_geo_mld_uit_functiepl', 'stm_scenario_mon', 'stm_status_melding_sap', 'stm_mon_nr_status_omschr', 'stm_mon_nr__statuscode', 'stm_mon_nr_status_wijzdd', 'stm_aanntpl_ddt', 'stm_geo_gst_uit_functiepl', 'stm_objectdl_code_gst', 'stm_objectdl_groep_gst', 'stm_progfh_in_ddt', 'stm_progfh_in_invoer_ddt', 'stm_progfh_gw_ddt', 'stm_progfh_gw_lwd_ddt', 'stm_progfh_hz', 'stm_veroorz_groep', 'stm_veroorz_code', 'stm_progfh_gw_duur', 'stm_progfh_gw_lwd_tijd', 'stm_progfh_gw_lwd_datum', 'stm_progfh_gw_datum', 'stm_arbeid', 'stm_functiepl_gst', 'stm_functiepl_mld', 'stm_mon_begintijd', 'stm_mon_begindatum', 'stm_aangelegd_tijd', 'stm_aangelegd_dd', 'stm_dir_betrok_tr', 'stm_evb', 'stm_mon_vhdsincident', 'stm_mon_eind_ddt', 'stm_afspr_aanvangddt', 'stm_effect', 'stm_veroorz_tekst_kort', 'stm_projnr', 'stm_wbi_nummer', 'stm_status_gebr', 'stm_standplaats', 'stm_actie', 'stm_hinderwaarde', 'stm_rec_toegev_ddt', 'stm_afsluit_tijd', 'stm_afsluit_dd', 'stm_afsluit_ddt', 'stm_akkoord_melding_jn', 'stm_fact_jn', 'stm_status_sapnaarmon', 'stm_akkoord_mon_toewijz', 'stm_controle_dd', 'stm_mon_eind_tijd', 'stm_mon_eind_datum', 'stm_reactie_duur', 'stm_afspr_aanvangtijd', 'stm_afspr_aanvangdd', 'stm_historie_toelichting', 'stm_schade_verhaalb_jn', 'stm_x_bron_arch_dt', 'stm_x_bron_bestandsnaam', 'stm_x_bron_publ_dt', 'stm_rapportage_jaar', 'stm_rapportage_maand', 'stm_sorteerveld', 'stm_afspr_func_hersteltijd', 'stm_afspr_func_hersteldd', 'stm_relatiervo', 'stm_relatiervo_volgend', 'stm_relatiervo_vorig', 'stm_schade_statusdatum', 'stm_schade_status_ga', 'stm_schadenr', 'stm_x_start_sessie_dt', 'stm_x_bk', 'stm_x_run_id', 'stm_x_actueel_ind', 'stm_x_vervallen_ind'])
print("Done")

Done


# Data cleaning:


De te bepalen welke variabelen het meest van toepassing zijn om te gebruiken kijken we naar welke
kolommen informatie geven over de tijd van de melding, monteur. Tijd wanneer de melding word opgelost.
Wat voor melding het is en info over de melding/storing, en de locatie. Zo zijn de volgende kolommen
het meest van toepassing:

Over soort melding
- stm_sap_meldtekst (melding omschrijving)
- stm_geo_mld (melding code) - nodig
- stm_oorz_code (oorzaak code) - nodig
- stm_prioriteit (prioriteit)

Over tijd:
- stm_sap_melddatum (datump melding) - nodig
- stm_sap_meldtijd (tijdstip melding) - nodig
- stm_aanngeb_dd (datum aannemer gebeld)
- stm_aanngeb_tijd (tijd aannemer gebeld)
- stm_aanntpl_dd (datum aannemer op plek)
- stm_aanntpl_tijd (tijd aannemer op plek)

- stm_fh_dd (Datum van functie herstel) - nodig
- stm_fh_tijd (Tijdstip van functieherstel) - nodig
- stm_sap_storeinddatum (Datum einde storing) - nodig
- stm_sap_storeindtijd (Tijdstip einde storing) - nodig
- stm_fh_duur (Duur van functiehersteltijd) - nodig

Over equipment:
- stm_equipm_nr_mld (equipment)
- stm_equipm_omschr_mld (equipment omschrijving)

Over locatie
- stm_km_van_mld (locatie van)
- stm_km_tot_mld (locatie tot)
- stm_vl_post (VL post) - nodig
- stm_contractgeb_mld(locatie melding) - nodig
- stm_pplg_van (PPLG van locatie)
- stm_pplg_naar (PPLG tot locatie)
- stm_dstrglp_van (Dienstregelpunt van locatie)
- stm_dstrglp_naar (Dienstregelpunt tot locatie)

Over prognose:
- stm_progfh_in_datum (Datum prognose)
- stm_progfh_in_tijd (Tijdstip FHT prognose)
- stm_progfh_in_invoer_dat (Invoer datum prognose)
- stm_progfh_in_invoer_tijd (Invoer tijdstip FHT prognose)
- stm_progfh_in_duur (Duur FHT)



In [128]:
#deze functie verwijdert alle rijen waar geen value op de gegeven kolommen zitten
def drop_rows_with_null(lst, df):
    dfisnull = df.isnull()[lst]
    matches = dfisnull[(dfisnull).any(axis=1)]
    return df.drop(matches.index)

In [133]:
# Verwijder de rijen die geen value bevatten in de benodigde kolommen
df_filtered = drop_rows_with_null(['stm_geo_mld', 'stm_sap_melddatum', 'stm_oorz_code', 'stm_sap_meldtijd',
                          'stm_fh_dd', 'stm_fh_tijd', 'stm_sap_storeinddatum', 'stm_sap_storeindtijd',
                          'stm_fh_duur', 'stm_vl_post', 'stm_contractgeb_mld'], df)
print(len(df))
print(len(df_filtered))
df_filtered

898526
132923


,Unnamed: 0,#stm_sap_meldnr,stm_mon_nr,stm_vl_post,stm_sap_meld_ddt,stm_sap_meldtekst_lang,stm_sap_meldtekst,stm_geo_mld,stm_equipm_nr_mld,stm_equipm_soort_mld,...,stm_fh_dd,stm_fh_tijd,stm_fh_duur,stm_sap_storeinddatum,stm_sap_storeindtijd,stm_oorz_tekst_kort,stm_pplg_van,stm_pplg_naar,stm_dstrglp_van,stm_dstrglp_naar
85,87,50053389,37.0,EHV,10/01/2006 16:11:00,Logboeknr RBV : 37 Tijd: 1611 VL-Post: ...,Logboeknr RBV : 37 Tijd: 1611 VL-P,618.0,10646141.0,SYSTBOVENL,...,10/01/2006,16:31:00,20.0,11/01/2006,10:33:00,Geen SMC-melding. Afwijzing. Bij het SMC,NaN,NaN,EHV,NaN
123,125,50053438,62.0,AH,15/01/2006 23:08:00,Logboeknr RBV : 62 Tijd: 2308 VL-Post: ...,Logboeknr RBV : 62 Tijd: 2308 VL-P,514.0,10201441.0,WISSEL,...,15/01/2006,23:20:00,12.0,15/01/2006,23:20:00,NaN,NaN,NaN,Nm,NaN
205,208,50053591,39.0,EHV,20/01/2006 14:32:00,Logboeknr RBV : 39 Tijd: 1432 VL-Post: ...,Logboeknr RBV : 39 Tijd: 1432 VL-P,515.0,10527838.0,WISSEL,...,20/01/2006,14:47:00,15.0,23/01/2006,11:39:00,Geen SMC-melding.,NaN,NaN,Vga,NaN
226,229,50053628,250.0,ZL,24/01/2006 20:20:00,Logboeknr RBV : 250 Tijd: 2020 VL-Post: ...,Logboeknr RBV : 250 Tijd: 2020 VL-P,25.0,10211614.0,OVERWEG,...,24/01/2006,20:30:00,10.0,24/01/2006,20:30:00,NaN,NaN,NaN,AMRI,NaN
240,243,50053652,42.0,RSD,25/01/2006 09:30:00,Logboeknr RBV : 42 Tijd: 0930 VL-Post: ...,Logboeknr RBV : 42 Tijd: 0930 VL-P,127.0,10621371.0,DETECTREIN,...,25/01/2006,09:44:00,14.0,30/01/2006,11:21:00,Geen SMC-melding.Geen onderzoek geweest.,NaN,NaN,Arn,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898413,908517,99003207,3669429.0,MT,21/05/2016 13:17:56,[2+] Mtr-Edng : Sp-Sb/TB t.o.b.s. alle ovw'en ...,[2+] Mtr-Edng : Sp-Sb/TB t.o.b.s. alle o,71.0,10621358.0,DETECTREIN,...,21/05/2016,14:35:00,175.0,21/05/2016,14:50:00,Gestoord object: SECTIE 913AT. Gestoord,MTR,NaN,MTR,EDN
898472,908576,99003486,1995699.0,RTD,10/05/2013 17:54:00,"Brd : deuren buis 2, 3, 4 en open tijdens tre...","Brd : deuren buis 2, 3, 4 en open tijden",116.0,10590754.0,TUNCONSTR,...,10/05/2013,19:38:00,104.0,10/05/2013,20:40:00,NaN,RTST,NaN,RTST,RLB
898473,908577,99003486,1995699.0,RTD,10/05/2013 17:54:00,"Brd : deuren buis 2, 3, 4 en open tijdens tre...","Brd : deuren buis 2, 3, 4 en open tijden",116.0,10590754.0,TUNCONSTR,...,10/05/2013,19:38:00,104.0,10/05/2013,20:40:00,NaN,RTST,NaN,RTST,RLB
898474,908578,99003486,1995699.0,RTD,10/05/2013 17:54:00,"Brd TTI: deuren buis 2, 3, 4 en open tijdens ...","Brd TTI: deuren buis 2, 3, 4 en open tij",116.0,10590754.0,TUNCONSTR,...,10/05/2013,19:38:00,104.0,10/05/2013,20:40:00,NaN,RTST,NaN,RTST,RLB


In [136]:
#uitgeschreven lijsten zodat ik ze niet steeds hoef over te typen

# melding
df[['stm_sap_meldtekst', 'stm_geo_mld', 'stm_oorz_code', 'stm_prioriteit']]
# tijd
df[['stm_sap_melddatum', 'stm_sap_meldtijd', 'stm_aanngeb_dd', 'stm_aanngeb_tijd', 'stm_aanntpl_dd', 'stm_aanntpl_tijd']]
df[['stm_fh_dd', 'stm_fh_tijd', 'stm_fh_duur', 'stm_sap_storeinddatum', 'stm_sap_storeindtijd']]
# equipment
df[['stm_equipm_omschr_mld', 'stm_equipm_nr_mld']]
# locatie
df[['stm_km_van_mld', 'stm_km_tot_mld', 'stm_vl_post', 'stm_contractgeb_mld', 'stm_pplg_van', 'stm_pplg_naar', 'stm_dstrglp_van', 'stm_dstrglp_naar']]
# prognose
df[['stm_progfh_in_datum', 'stm_progfh_in_tijd', 'stm_progfh_in_invoer_dat', 'stm_progfh_in_invoer_tijd', 'stm_progfh_in_duur']]


,stm_progfh_in_datum,stm_progfh_in_tijd,stm_progfh_in_invoer_dat,stm_progfh_in_invoer_tijd,stm_progfh_in_duur
0,NaN,NaN,NaN,NaN,NaN
1,NaN,00:00:00,NaN,00:00:00,99999999.0
2,NaN,00:00:00,NaN,00:00:00,99999999.0
3,NaN,00:00:00,NaN,00:00:00,99999999.0
4,NaN,00:00:00,NaN,00:00:00,99999999.0
...,...,...,...,...,...
898521,11/05/2013,09:35:00,11/05/2013,09:26:18,30
898522,11/05/2013,11:00:00,11/05/2013,10:16:27,52
898523,11/05/2013,11:00:00,11/05/2013,10:16:27,52
898524,20/08/2016,16:00:00,20160820,15:16:29,75
